In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img , ImageDataGenerator
from tensorflow.nn import relu , softmax
from tensorflow.keras.layers import Dense , Conv2D , MaxPool2D , Flatten
from tensorflow.keras import Model

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")
import numpy as np

In [16]:
class convBlock(Model):
    def __init__(self , filters , kernal ,repetitions):
        super(convBlock , self).__init__()
        self.filters = filters
        self.kernal = kernal
        self.repetitions = repetitions
        for i in range(self.repetitions):
            vars(self)["conv_2D_%d"%(i)] = Conv2D(self.filters , self.kernal , padding='same' , activation=relu)
        self.maxpool = MaxPool2D(strides=2)

    def call(self , inputs):
        x = vars(self)["conv_2D_%d"%(0)](inputs)
        for i in range(1 , self.repetitions):
            x = vars(self)["conv_2D_%d"%(i)](x)
        return self.maxpool(x)
    
class VGG16(Model):
    def __init__(self , dense = 128 , classes = 1 , activation = "sigmoid"):
        super(VGG16 , self).__init__()
        self.block_a = convBlock(64 , 3 ,2)
        self.block_b = convBlock(128 , 3 ,2)
        self.block_c = convBlock(256 , 3 ,2)
        self.block_d = convBlock(512 , 3 ,3)
        self.block_e = convBlock(512 , 3 ,3)
        self.flatten = Flatten()
        self.dense1 = Dense(dense , activation=relu)
        self.dense2 = Dense(classes , activation=tf.keras.activations.get(activation))

    def call(self , inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)

        return x
    
vgg_model = VGG16()
#vgg_model.compile(optimizer='adam' , loss=tf.keras.losses.binary_crossentropy , metrics=['acc'])

In [17]:
val_data_gen = ImageDataGenerator(rescale=(1.0 / 255.0))

In [18]:
val_dir = "./Validation/"

In [19]:
val_data = val_data_gen.flow_from_directory(val_dir , target_size=(224 , 224) , class_mode='binary')

Found 11649 images belonging to 2 classes.


In [20]:
shp = vgg_model(val_data[0][0][:5])

In [21]:
vgg_model.summary()

Model: "vg_g16_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block_10 (convBlock)    multiple                  0         
_________________________________________________________________
conv_block_11 (convBlock)    multiple                  0         
_________________________________________________________________
conv_block_12 (convBlock)    multiple                  0         
_________________________________________________________________
conv_block_13 (convBlock)    multiple                  0         
_________________________________________________________________
conv_block_14 (convBlock)    multiple                  0         
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  321

In [22]:
vgg_model.weights

[<tf.Variable 'vg_g16_2/dense_4/kernel:0' shape=(25088, 128) dtype=float32, numpy=
 array([[ 0.00648436,  0.00619326, -0.00162265, ..., -0.00851444,
         -0.01013879,  0.01008122],
        [-0.00863614, -0.01411449,  0.00649536, ..., -0.01391899,
          0.01171307, -0.00944411],
        [-0.01361583, -0.01107179, -0.00499182, ..., -0.01251624,
         -0.0101733 , -0.00493423],
        ...,
        [ 0.01059896, -0.01504976, -0.00809491, ...,  0.00105818,
         -0.01160358, -0.00752967],
        [-0.00545881, -0.00278217,  0.00114546, ...,  0.00027408,
          0.01439839,  0.00298457],
        [ 0.00036846,  0.01488451,  0.00703789, ...,  0.01466557,
          0.01020601, -0.00013055]], dtype=float32)>,
 <tf.Variable 'vg_g16_2/dense_4/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [23]:
vgg_model.load_weights("./my_new_model_weight.h5")

In [24]:
vgg_model.compile(loss=tf.keras.losses.binary_crossentropy , optimizer = 'adam' , metrics=['acc'])

In [25]:
vgg_model.weights

[<tf.Variable 'vg_g16_2/dense_4/kernel:0' shape=(25088, 128) dtype=float32, numpy=
 array([[-0.01550002,  0.01119892, -0.01080055, ..., -0.01728782,
          0.00442336, -0.00531013],
        [-0.01227118,  0.00818758, -0.01316513, ...,  0.00317032,
          0.00693901,  0.00328031],
        [-0.00326071, -0.01446931, -0.00537181, ...,  0.00168268,
         -0.00941756,  0.00834451],
        ...,
        [-0.0074073 ,  0.00120761, -0.0021443 , ..., -0.00246696,
         -0.01016922,  0.00463643],
        [ 0.00634986, -0.00090727, -0.01603296, ...,  0.00887409,
          0.01442252, -0.00491507],
        [ 0.00600276,  0.00364838, -0.00731489, ..., -0.00624161,
          0.0027818 , -0.00232937]], dtype=float32)>,
 <tf.Variable 'vg_g16_2/dense_4/bias:0' shape=(128,) dtype=float32, numpy=
 array([-0.00560776,  0.        , -0.00600271, -0.0053319 , -0.00068837,
        -0.00560713, -0.00508057, -0.00689821, -0.00598982, -0.00599122,
        -0.00600461, -0.01386098, -0.00529654, -0.005

In [27]:
val_data[0][1][:5]

array([1., 1., 1., 0., 0.], dtype=float32)

In [28]:
vgg_model.predict(val_data[0][0][:5])

array([[0.9996989 ],
       [0.99996334],
       [0.99999905],
       [0.9999994 ],
       [0.99998593]], dtype=float32)